# <center><font color="blue">Modelación Cinemática y Aplicación de Algoritmos de Control Avanzado en Webots</font></center>


### <center>Luis F Recalde</center><center><font size="2"><i>Artil Robotics</i></font></center>

#### <center>Santo Domingo de los Colorados, 21 de Septiembre de 2020</center>

# <center><font color="red">Controlador No Lineal Basado en Lyapunov</font></center>

<div style="text-align: justify">A continuacion se presenta el modelo a utilizar en esta seccion, el cual se desarrollo en la practica anterior.</div>

In [9]:
b = 56

In [10]:
c = a + b

In [11]:
print(c)

79
